<a href="https://colab.research.google.com/github/srinivasasmaveric/dash-technical-charting/blob/master/DataPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import re

In [0]:
df_final=pd.read_csv('./BIDATAHIST.csv')

In [0]:
from sklearn.cluster import KMeans
df_clusters_master=df_final[['PRICE','DAYSPERMILLION']].astype(np.float64)
X=df_clusters_master.values
km=KMeans(n_clusters=5)
km.fit(X)
df_clusters_master['Cluster']=pd.Series(km.labels_)

In [0]:
# Prediction

In [0]:
X_df=df_final.drop(columns=['DAYSPERMILLION','REGION','INVOICE_DATE','CUSTOMER_NAME','RECEIPT_DATE','Year','Inflow_Days','Month_diff','Quarter_diff'])
y_df=df_final[['DAYSPERMILLION','Month_diff','Quarter_diff']]

In [0]:
X=X_df.values
y_dpm=y_df[['DAYSPERMILLION']].values
y_md=y_df[['Month_diff']].values
y_qd=y_df[['Quarter_diff']].values

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [0]:
X_tr_dp,X_ts_dp,y_tr_dp,y_ts_dp=train_test_split(X,y_dpm,test_size=0.25)

In [0]:
rf=RandomForestRegressor()

In [10]:
rf.fit(X_tr_dp,y_tr_dp)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [11]:
print(rf.score(X_tr_dp,y_tr_dp))
print(rf.score(X_ts_dp,y_ts_dp))

0.898538080267174
0.46161975278229905


In [0]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

In [13]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 2000, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [5, 10, 15, 20, 25, 30, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [500, 1000, 1500, 2000]}


In [14]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_tr_dp, y_tr_dp)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 54.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 92.5min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [23]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1500}

In [0]:
rf_opt=RandomForestRegressor(bootstrap= True, 
                              max_depth= 20, 
                              max_features= 'auto', 
                              min_samples_leaf= 1, 
                              min_samples_split= 2, 
                              n_estimators= 1500)

In [19]:
rf_opt.fit(X_tr_dp,y_tr_dp)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [20]:
print(rf_opt.score(X_tr_dp,y_tr_dp))
print(rf_opt.score(X_ts_dp,y_ts_dp))

0.9021403588757001
0.4767831161918138
